<a href="https://colab.research.google.com/github/chunter3/Information_Retrieval_Projects/blob/master/Wildcard_Non_Wildcard_Query_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import re
%matplotlib inline

In [2]:
# Problem 1 (start)

In [ ]:
# Loading the state_union sample dataset from the nltk library

nltk.download('punkt')
nltk.download('state_union')
from nltk.corpus import state_union
state_union.fileids()  # 65 documents w/in the state_union corpus

In [4]:
# Inverted index function

def InvertedIndex(corpus):
  invertedIndex = {}
  docID = 0
  for textFile in corpus:
    text = state_union.sents(textFile)
    docID+=1
    for sentence in text:   
      joinTerms = ' '.join(sentence)        
      processSent = (re.sub('[^A-Za-z0-9]+',' ', joinTerms)).split()
      for word in processSent:
         if word in invertedIndex and docID not in invertedIndex[word]:
           invertedIndex[word].append(docID)
      else:
        invertedIndex[word] = [docID]
  return invertedIndex

In [ ]:
InvertedIndex(state_union.fileids())  # 3837 entries in the inverted index; 16097 tokens in total

In [6]:
# Problem 1 (end)

In [7]:
# Problem 2 (start)

In [8]:
# Intersect auxilary function; called w/in primary function WCQSearch

def Intersect(p1,p2):
    intersect = [term for term in p1 if term in p2]
    if intersect == []:
      return p1
    return intersect

In [9]:
# RelevantBigrams auxilary function; called w/in primary function WCQSearch

def RelevantBigrams(bigrams,query):
  relevantBigrams = []
  partitionQuery = query.partition("*")
  for partition in partitionQuery:
    if partition == "*" or "":
      continue
    for bigram in bigrams:
      if len(bigram) == 1:
        relevantBigrams.append(bigram)
        continue
      elif bigram in partition:
        relevantBigrams.append(bigram)
  return list(set(relevantBigrams))

In [10]:
# Wildcard query search function for single term wildcard queries

def WCQSearch(query,index): 
  if "*" not in query or " " in query:                      # Return immediately if query isn't a wildcard
    return "Query is either not a wildcard or multi termed. Goodbye"
  finalDict = {}           # The final dictionary to be returned
  bigramDict = {}          
  bigrams = []
  wcQuery = query          # The initial, untouched query; used during post-filtering phase

#--------------------------------Bigram dictionary creation phase

  if "*" == query[0]:
    query = query.replace("*","") + "$"
  elif "*" == query[-1]:
    query = "$" + query.replace("*","")
  else:
    query = "$" + query.replace("*","") + "$" 
  for x in range (1,len(query)):
      bigramDict[(query[x-1:x+1]).replace("$","")] = []     # Adds bigrams as keys to bigramDict
      bigrams.append(query[x-1:x+1]) 
  for bigram in bigrams:
    if "$" in bigram[0]:                                    # Handles bigrams where $ is in the front (e.g., "$p")
      for key in index.keys():                             
        if bigram[1] == key[0]:
          bigramDict[bigram[1]].append(key)
    elif "$" in bigram[1]:                                  # Handles bigrams where $ is in the back (e.g., "p$")
      for key in index.keys():
        if bigram[0] == key[-1]:
          bigramDict[bigram[0]].append(key)
    else:
      for key in index.keys():                               # Handles bigrams w/out $s
        if bigram in key:                 
          bigramDict[bigram].append(key)
  bigrams.clear()
  for x in range (1,len(query)):                             # Creates new bigram list w/out $s
    bigrams.append((query[x-1:x+1]).replace("$",""))
  
#--------------------------------Bigram dictionary created; post-filtering phase

  relevantBigrams = RelevantBigrams(bigrams,wcQuery)          # Generates a list of the most significant bigrams based on query
  partitionQuery = wcQuery.partition("*")

# ------------------------------------------------
  if partitionQuery[2] == "" and wcQuery[-1:] == "*":
    for key in index.keys():
      if partitionQuery[0] == key[:len(partitionQuery[0])]:
        finalDict[key] = index[key]                          
    return finalDict
                                                              # Handles terminal queries (e.g., *n or s*)
  elif partitionQuery[0] == "" and wcQuery[0] == "*":
    for key in index.keys():
      if partitionQuery[2] == key[(-1 * (len(partitionQuery[2]))):]:
        finalDict[key] = index[key]
    return finalDict
# --------------------------------------------------

  intersect = []
  intersect = Intersect(bigramDict[relevantBigrams[0]],bigramDict[relevantBigrams[1]])
  del relevantBigrams[:2]
  for bigram in relevantBigrams:
      p1 = Intersect(intersect,bigramDict[bigram])
      intersect = p1
  if partitionQuery[0] == "" or partitionQuery[2] == "":                                 
    for term in intersect:
      finalDict[term] = index[term]
  for term in intersect:
        if partitionQuery[0] == term[:len(partitionQuery[0])] and partitionQuery[2] == term[(-1 * (len(partitionQuery[2]))):]:
          finalDict[term] = index[term]
  return finalDict

In [21]:
# Let's examine a single term wildcard query to verify the function's accuracy
wcQuery = "un*y" 
index = InvertedIndex(state_union.fileids())
WCQSearch(wcQuery,index)

{'unanimity': [18],
 'uncertainty': [59, 64, 65],
 'underway': [41, 47],
 'unduly': [32, 53],
 'unevenly': [13],
 'unity': [26, 28, 35, 36, 37, 41, 42, 43, 50, 60, 61, 62, 64],
 'unnecessary': [36, 37, 38, 39, 53, 54, 55]}

In [22]:
# Note that the following is returned for multi term or non-wildcard queries
nwcQuery = "Hello"
WCQSearch(nwcQuery,index)

'Query is either not a wildcard or multi termed. Goodbye'

In [13]:
# Problem 2 (end)

In [14]:
# Problem 3 (start)

In [15]:
# Non-wildcard query search function for single term non-wildcard queries

def NWCQSearch(query,index):
  if "*" in query or " " in query:                      
    return "Query is either a wildcard or multi termed. Goodbye"
  finalDict = {}           
  bigramDict = {}          
  bigrams = []
  for key in index.keys():
    if query == key:              # If query is spelled correctly and in inverted index, return corresponding postings list
      finalDict[key] = index[key]
      return finalDict
  nwcQuery = query        
  if "*" == query[0]:
    query = query.replace("*","") + "$"
  elif "*" == query[-1]:
    query = "$" + query.replace("*","")
  else:
    query = "$" + query.replace("*","") + "$" 
  for x in range (1,len(query)):
      bigramDict[(query[x-1:x+1]).replace("$","")] = []     
      bigrams.append(query[x-1:x+1]) 
  for bigram in bigrams:
    if "$" in bigram[0]:                                   
      for key in index.keys():                             
        if bigram[1] == key[0]:
          bigramDict[bigram[1]].append(key)
    elif "$" in bigram[1]:                                  
      for key in index.keys():
        if bigram[0] == key[-1]:
          bigramDict[bigram[0]].append(key)
    else:
      for key in index.keys():                              
        if bigram in key:                 
          bigramDict[bigram].append(key)
  bigrams.clear()
  for x in range (1,len(query)):                            
    bigrams.append((query[x-1:x+1]).replace("$",""))

 #-------------------------------------Get edit distances        

  editDists = []                    # 2D array of edit distances
  minDists = []                     # 1D array of the minimum edit distances
  for bigram in bigrams:
    editDist = []                   # 1D array of edit distances
    for term in bigramDict[bigram]:
      editDist.append(nltk.edit_distance(nwcQuery,term))
    editDists.append(editDist)
  for lst in editDists:
    minDists.append(min(lst))

#------------------------------------Edit distances obtained; generating suggestions

  minDist = min(minDists)
  suggestions = []           # A list of potential spelling suggestions
  pointer = -1                # Tracks list position in editDists
  for lst in editDists:
    pointer+=1
    for x in range(0,len(lst)):
      if lst[x] == minDist:
        suggestions.append(bigramDict[bigrams[pointer]][x])
  suggestions = list(set(suggestions))        # Removing duplicate suggestions
  if len(suggestions) == 1:
      return "No search results. Did you mean '" + suggestions[0] + "'?"
  elif len(suggestions) == 2:
      return "No search results. Did you mean '" + suggestions[0] + "' or '" + suggestions[1] + "'?"
  else:
      finalMsg = ""
      for x in range(0,len(suggestions)):
        if x == (len(suggestions) - 1):
          finalMsg = finalMsg + "or '" + suggestions[x] + "'?"
          break
        finalMsg = "'" + suggestions[x] + "', " + finalMsg

  return "No search results. Did you mean " + finalMsg

In [23]:
# Let's test a correctly spelled single term non-wildcard query

nwcquery = "population"
NWCQSearch(nwcquery,index)

{'population': [24, 25, 27, 28, 36, 40, 55, 62]}

In [24]:
# Let's test a misspelled single term non-wildcard query

nwcquery = "cream"
NWCQSearch(nwcquery,index)

"No search results. Did you mean 'dream'?"

In [25]:
# Additionally, multiple suggestions are possible

nwcquery = "yost"
NWCQSearch(nwcquery,index)

"No search results. Did you mean 'cost', 'most', or 'lost'?"

In [26]:
# Note that the following is returned for multi term or wildcard queries

nwcquery = "f*l"
NWCQSearch(nwcquery,index)

'Query is either a wildcard or multi termed. Goodbye'

In [20]:
# Problem 3 (end)